# to do
- test set has to be put in a goof form
- the promointerval was taken out until I can put it in a good form as well
- how to change the arrays inside an array as individual columns (otherwise I think SVM etc will not work)

In [201]:
import numpy as np
import pandas 
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn import preprocessing

In [202]:
store = pandas.read_csv('store.csv', sep = ',')
train_initial = pandas.read_csv('train_v2.csv',dtype={"StateHoliday": str})
test = pandas.read_csv('test_v2.csv', sep = ",") 

#take out the sales (we don't have this info in the test)
sales_train = train_initial['Sales']
train = train_initial.drop('Sales', axis=1)


#take out days were it's closed
sales_train = pandas.Series.to_frame(sales_train)
train = train[train.Customers != 0]
sales_train = sales_train[sales_train.Sales != 0]

In [203]:
test

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,555,1,1,0,1
1,2,5,2015-07-31,625,1,1,0,1
2,3,5,2015-07-31,821,1,1,0,1
3,4,5,2015-07-31,1498,1,1,0,1
4,5,5,2015-07-31,559,1,1,0,1
5,6,5,2015-07-31,589,1,1,0,1
6,7,5,2015-07-31,1414,1,1,0,1
7,8,5,2015-07-31,833,1,1,0,1
8,9,5,2015-07-31,687,1,1,0,1
9,10,5,2015-07-31,681,1,1,0,1


In [204]:
# take out the stores with no sales - 
# train = train[train.Sales != 0] |||| NOT SURE IF GOOD IDEA

# put the 2 types of data together
df = train.join(store.set_index('Store'), on='Store')
test = test.join(store.set_index('Store'), on='Store')

# extract the date as day/month/year/weak of the year
df['Date']  = pandas.to_datetime(df['Date'], errors='coerce')
df['Year']  = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.weekofyear

test['Date']  = pandas.to_datetime(test['Date'], errors='coerce')
test['Year']  = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.weekofyear

In [205]:
df

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear
0,1,5,2015-01-30,616,1,1,0,0,c,a,...,9.0,2008.0,0,NaN,NaN,NaN,2015,1,30,5
1,2,5,2015-01-30,624,1,1,0,0,a,a,...,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,1,30,5
2,3,5,2015-01-30,678,1,1,0,0,a,a,...,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,1,30,5
3,4,5,2015-01-30,1632,1,1,0,0,c,c,...,9.0,2009.0,0,NaN,NaN,NaN,2015,1,30,5
4,5,5,2015-01-30,617,1,1,0,0,a,a,...,4.0,2015.0,0,NaN,NaN,NaN,2015,1,30,5
5,6,5,2015-01-30,692,1,1,0,0,a,a,...,12.0,2013.0,0,NaN,NaN,NaN,2015,1,30,5
6,7,5,2015-01-30,1077,1,1,0,0,a,c,...,4.0,2013.0,0,NaN,NaN,NaN,2015,1,30,5
7,8,5,2015-01-30,747,1,1,0,0,a,a,...,10.0,2014.0,0,NaN,NaN,NaN,2015,1,30,5
8,9,5,2015-01-30,643,1,1,0,0,a,c,...,8.0,2000.0,0,NaN,NaN,NaN,2015,1,30,5
9,10,5,2015-01-30,602,1,1,0,0,a,a,...,9.0,2009.0,0,NaN,NaN,NaN,2015,1,30,5


In [206]:
# combine all to make sure test and data get the same dummies 
number_train = df.shape[0]
number_test = test.shape[0]
frames = [df,test]
result = pandas.concat(frames)


In [207]:
result = result.fillna(-1)

In [208]:
def month_difference (comp_year, our_year, comp_month, our_month):
    difference = our_month - comp_month + 12*(our_year-comp_year)
    return difference
    

In [209]:
result['CompetitionOpenSinceHowLong'] = month_difference (result['CompetitionOpenSinceYear'],result['Year'],
                                                       result['CompetitionOpenSinceMonth'],result['Month'])

In [210]:
result.head()

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpenSinceHowLong
0,1,5,2015-01-30,616,1,1,0,0,c,a,...,2008.0,0,-1.0,-1.0,-1,2015,1,30,5,76.0
1,2,5,2015-01-30,624,1,1,0,0,a,a,...,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,1,30,5,86.0
2,3,5,2015-01-30,678,1,1,0,0,a,a,...,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,1,30,5,97.0
3,4,5,2015-01-30,1632,1,1,0,0,c,c,...,2009.0,0,-1.0,-1.0,-1,2015,1,30,5,64.0
4,5,5,2015-01-30,617,1,1,0,0,a,a,...,2015.0,0,-1.0,-1.0,-1,2015,1,30,5,-3.0


In [211]:
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
result.StateHoliday.replace(mappings, inplace=True)
result.StoreType.replace(mappings, inplace=True)
result.Assortment.replace(mappings, inplace=True)

In [213]:
result.drop(['PromoInterval','Date','Day','CompetitionOpenSinceMonth','Date'], axis=1, inplace=True)

        

ValueError: labels ['PromoInterval' 'Date' 'Day' 'CompetitionOpenSinceMonth' 'Date'] not contained in axis

In [214]:
#result.drop(['StoreType','Assortment'], axis=1, inplace=True)

In [215]:
df = result.iloc[:number_train,:]
test = result.iloc[number_train:,:]

In [216]:
test.head()

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,WeekOfYear,CompetitionOpenSinceHowLong
0,1,5,555,1,1,0,1,3,1,1270.0,2008.0,0,-1.0,-1.0,2015,7,31,82.0
1,2,5,625,1,1,0,1,1,1,570.0,2007.0,1,13.0,2010.0,2015,7,31,92.0
2,3,5,821,1,1,0,1,1,1,14130.0,2006.0,1,14.0,2011.0,2015,7,31,103.0
3,4,5,1498,1,1,0,1,3,3,620.0,2009.0,0,-1.0,-1.0,2015,7,31,70.0
4,5,5,559,1,1,0,1,1,1,29910.0,2015.0,0,-1.0,-1.0,2015,7,31,3.0


In [217]:
df = df.astype(np.float32)
test = test.astype(np.float32)

In [218]:

#put the data as a numpy array
data_train = df.values
#sales_train = sales_train_train.values
sales_train = pandas.DataFrame.as_matrix(sales_train)


In [219]:
sales_train = sales_train.reshape(len(sales_train),)

In [220]:
X = data_train

In [221]:
y = sales_train

In [222]:
randomforest = RandomForestRegressor(n_estimators=250, max_features = 'auto')
randomforest.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=250, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

0.2246684265967324   10
0.19790389121535615  50
0.19413401753256496  100
0.19102393447665608  250
0.1903307154295898   500
0.19031775372145007  1000
0.19000211033516165  50000
0.19000798708563635  10000

#-------------


In [223]:
prediction_of_sales = randomforest.predict(test)

In [180]:
prediction_in_sample = randomforest.predict(X)

In [226]:
for i in range (number_test) :
    if test['Customers'].iloc[i] == 0 :
        prediction_of_sales[i] = 0

In [183]:
from math import sqrt
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y[i] != 0:
            summ += (1-y_pred[i]/y[i])**2
    return sqrt(summ/len(y))

In [184]:
rmspe (sales_train, prediction_in_sample)

0.19416226504547487

In [227]:
submission = pandas.DataFrame()
submission['Sales'] = prediction_of_sales
# submission.Sales = submission.Sales.astype(int)
cols = ['Id','Sales']
submission['Id'] = submission.index + 1
submission = submission[cols]
submission.to_csv('submission.csv', index=False)